In [8]:
import os
import shutil
import tempfile
import matplotlib.pyplot as plt
from PIL import Image
import torch
import numpy as np
from sklearn.metrics import classification_report

from monai.apps import download_and_extract
from monai.config import print_config
from monai.metrics import ROCAUCMetric
from monai.networks.nets import DenseNet121
from monai.transforms import (
    Activations,
    EnsureChannelFirst,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    ToTensor,
)
from monai.data import Dataset, DataLoader
from monai.utils import set_determinism

print_config()

MONAI version: 1.3.0
Numpy version: 1.26.4
Pytorch version: 2.3.0+cpu
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 865972f7a791bf7b42efbcd87c8402bd865b329e
MONAI __file__: c:\Users\DC\Desktop\Monai\.ven\lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: 5.3.0
Nibabel version: 5.2.1
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: 1.13.0
Pillow version: 10.3.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: 4.7.3
TorchVision version: 0.18.0+cpu
tqdm version: 4.66.4
lmdb version: 1.4.1
psutil version: 5.9.8
pandas version: 2.2.2
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: 4.21.3
mlflow version: 2.12.2
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/

In [18]:
 pip install wget  # Install wget 

Note: you may need to restart the kernel to use updated packages.


In [17]:
!python -m wget https://www.dropbox.com/s/5wwskxctvcxiuea/MedNIST.tar.gz


Saved under MedNIST.tar.gz


In [23]:
# unzip the '.tar.gz' file to the current directory
import tarfile
datafile = tarfile.open("MedNIST.tar.gz")
datafile.extractall()
datafile.close()


### Read image filenames from the dataset folders

First of all, check the dataset files and show some statistics. There are 6 folders in the dataset: Hand, AbdomenCT, CXR, ChestCT, BreastMRI, HeadCT, which should be used as the labels to train our classification model.


In [26]:
data_dir = './MedNIST'
class_names = sorted([x for x in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir,x))])
num_class = len(class_names)
image_files = [[os.path.join(data_dir, class_name, x)
                for x in os.listdir(os.path.join(data_dir, class_name))]
               for class_name in class_names]

image_file_list = []
image_label_list = []
for i, class_name in enumerate(class_names):
    image_file_list.extend(image_files[i])
    image_label_list.extend([i]*len(image_files[i]))
num_total = len(image_label_list)
image_width, image_height = Image.open(image_file_list[i]).size

print('tTotal image count:', num_total)
print('Image Dimensions:', image_width, "x", image_height)
print("Label Names:", class_names)
print("Label Counts:", [len(image_files[i]) for i in range(num_class)])




tTotal image count: 58954
Image Dimensions: 64 x 64
Label Names: ['AbdomenCT', 'BreastMRI', 'CXR', 'ChestCT', 'Hand', 'HeadCT']
Label Counts: [10000, 8954, 10000, 10000, 10000, 10000]
